

- cf. [Google Colab で LangGraph を試す](https://note.com/npaka/n/n053a3cb78311?sub_rt=share_h)

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

assert "OPENAI_API_KEY" in os.environ
assert "TAVILY_API_KEY" in os.environ

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

In [3]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)

In [5]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

In [6]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


workflow = StateGraph(AgentState)

In [7]:
import json
from langgraph.prebuilt import ToolInvocation
from langchain_core.messages import FunctionMessage


# agentノードの次に呼び出されるノードを決定する関数の準備
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]

    # function_call がない場合は終了
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # そうでない場合は続行
    else:
        return "continue"


# agentノードの処理を定義する関数の準備
def call_model(state):
    messages = state["messages"]

    # モデルの呼び出し
    response = model.invoke(messages)

    # メッセージリストに追加するメッセージを返す
    return {"messages": [response]}


# actionノードの処理を定義する関数の準備
def call_tool(state):
    messages = state["messages"]
    last_message = messages[-1]

    # ツールの呼び出し
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        ),
    )
    print(f"[DEBUG] {action=}")
    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=action.tool)

    # メッセージリストに追加するメッセージを返す
    return {"messages": [function_message]}

In [8]:
from langgraph.graph import END

# グラフに2つのノードを追加
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# agentをエントリポイントとして指定
workflow.set_entry_point("agent")

# agentノードに条件付きエッジを追加
workflow.add_conditional_edges(
    "agent",  # 開始ノード
    should_continue,  # 呼び出されるノードを決定する関数
    {"continue": "action", "end": END},  # actionノードに遷移  # 終了
)

# actionノードからagentノードへのノーマルエッジを追加
workflow.add_edge("action", "agent")

# コンパイル
app = workflow.compile()

In [13]:
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.function import FunctionMessage
from langchain_core.messages.human import HumanMessage


def print_response(response: dict):
    for itm in response["messages"]:
        if isinstance(itm, HumanMessage):
            print("Human:", itm.content)
        elif isinstance(itm, FunctionMessage):
            print("Function:", itm.content)
        elif isinstance(itm, AIMessage):
            print("AI:", itm.content)
    return

In [14]:
from langchain_core.messages import HumanMessage

# 質問応答
inputs = {"messages": [HumanMessage(content="日本の首都は？")]}
response = app.invoke(inputs)
print_response(response)

Human: 日本の首都は？
AI: 東京です。


In [15]:
from langchain_core.messages import HumanMessage

# 質問応答
inputs = {"messages": [HumanMessage(content="日本で一番高い山の名前と標高は？")]}
response = app.invoke(inputs)
print_response(response)

[DEBUG] action=ToolInvocation(tool='tavily_search_results_json', tool_input={'query': '日本で一番高い山の名前と標高'})
Human: 日本で一番高い山の名前と標高は？
AI: 


TypeError: string indices must be integers

In [12]:
# ストリーミング
inputs = {"messages": [HumanMessage(content="日本で一番高い山の名前と標高を調べた上で、その山の歴史について説明してください")]}
for output in app.stream(inputs):
    for key, tkn in output.items():
        print(f"Output from node '{key}': ", end="")
        for msg in tkn["messages"]:
            print(msg.content)
    print("\n---\n")

Output from node 'agent': 

---

[DEBUG] action=ToolInvocation(tool='tavily_search_results_json', tool_input={'query': '日本で一番高い山の名前と標高'})
Output from node 'action': [{'url': 'https://www.日本百名山.net/column/takasa.html', 'content': '日本の山を標高が高い順にランキング形式で紹介。1位はお馴染み富士山ですが2位以降の山は果たして・・・ 1.富士山\u30003,776m 撮影：chantatsu  30.大天井岳\u30002,921m 大天井岳は北アルプスにある標高2,922mの山で、常念山脈の最高峰です。 関連記事 日本百名山一覧 深田久弥の著書『日本百名山』で紹介されている100山を一覧で紹介。 富士山撮影スポット  日本の標高の高い山ランキングBEST30｜日本百名山.net 日本百名山.net＞コラム＞日本の標高の高い山ランキング BEST30 日本の標高の高い山ランキング BEST30  撮影：chantatsu 富士山は日本で一番高い山として誰もが知っている山です。日本の象徴として海外の人にも知れ渡っており、2013年6月26日には世界文化遺産にも指定されました。 2.北岳\u30003,193m富士山は日本で一番高い山として誰もが知っている山です。日本の象徴として海外の人にも知れ渡っており、2013年6月26日には世界文化遺産にも指定されました。 富士山の詳細; 2.北岳 3,193m. 北岳は南アルプス最高峰で、日本で2番目に高い山です。'}]

---

Output from node 'agent': 日本で一番高い山は富士山です。富士山の標高は3,776メートルです。富士山は日本の象徴として知られており、世界文化遺産にも指定されています。

また、日本で2番目に高い山は北岳で、標高は3,193メートルです。北岳は南アルプスの最高峰であり、日本でも有名な山の一つです。

富士山の歴史については、古くから日本の信仰の対象となってきました。富士山は山岳信仰の対象として、修行や登山が行われてきました。また、富士山は絵画や文学の題